# Mechanisms of Action Project Submission

Team Members: Abhi Gandhi, Jeff Won, Shashank Saurav, Shengebo Zhang, Faraz Khoshbakhtian  


Main contributor to this document: Shengbo Zhang

This document outlines the steps taken by our group to approach the Mechanisms of Action competition. This document presents are original approach whereas another document presents the challanger approach. 

## Importing the needed libraries

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from operator import add

### This part is to run pyspark locally
import findspark  # Get rid of this in DataBricks
# findspark.init('/opt/spark-3.0.1')  # Get rid of this in DataBricks #faraz: you can remove the parameter. it only worked like this for me
findspark.init()
########################################

import pyspark
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10, sqrt
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.functions import vector_to_array


from tqdm import tqdm

from pyspark.ml.regression import DecisionTreeRegressor
from scv import StratifiedCrossValidator  # This file is for crossvalidation. Please see scv.py for more information

from pyspark.ml.classification import DecisionTreeClassifier
import os

## Seeting up spark, contexts and configurations

In [2]:
config = SparkConf().setAll([('spark.executor.memory', '50g'), ('spark.executor.cores', '8'), ('spark.cores.max', '8'), ('spark.driver.memory','8g')])
config.setAppName("proj")
config.set("spark.dynamicAllocation.minExecutors", "1");
config.set("spark.dynamicAllocation.maxExecutors", "8");
config.set("spark.dynamicAllocation.initialExecutors", "2"); # the number must be between the min and max
sc = SparkContext(conf=config)  # start a new sc with the current config
spark = SparkSession(sc)
sqlc=SQLContext(sc)
print(sc.getConf().getAll())  # print all the configuration

[('spark.dynamicAllocation.minExecutors', '1'), ('spark.dynamicAllocation.initialExecutors', '2'), ('spark.driver.port', '39545'), ('spark.executor.id', 'driver'), ('spark.driver.host', '192.168.176.164'), ('spark.rdd.compress', 'True'), ('spark.driver.memory', '8g'), ('spark.serializer.objectStreamReset', '100'), ('spark.cores.max', '8'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.dynamicAllocation.maxExecutors', '8'), ('spark.executor.cores', '8'), ('spark.app.name', 'proj'), ('spark.app.id', 'local-1608141748744'), ('spark.ui.showConsoleProgress', 'true'), ('spark.executor.memory', '50g')]


In [8]:
from pyspark.ml.classification import GBTClassifier


In [9]:
obj = GBTClassifier()

In [15]:
obj.getMaxIter()

20

## Reading and Preprocessing the Train Data

### Reading and Joining dataframes

First, we need to read the features and labels for the training data, get rid of controll cases and join all the training data into a singular Spark dataframe.

In [3]:
# This is for being able to store the data in github and concatenation on local computer
!cat train_features_*.csv > train_feats.csv

In [3]:
# Train Features Dataset
df_train = spark.read.csv('train_feats.csv', header='true', inferSchema= 'true')   # path in HDFS file system

In [4]:
# Labels Dataset
df_label = spark.read.csv('train_targets_scored.csv', header='true', inferSchema= 'true')

In [5]:
# join all data together
df = df_train.join(df_label, on=['sig_id'], how='left_outer')  

In [6]:
# Dropping all the control cases since the test data also comes with control/case flag
df = df.filter(df.cp_type == 'trt_cp')
df = df.drop('cp_type')

### One-Hot Encoding for Categorical features

Second, we need to **One-Hot Encode** our categorical features. 

In [7]:
# String indexer for cp_dose
indexer = StringIndexer(inputCol="cp_dose", outputCol="cp_dose_cat")
# index cp_dose in data
df1 = indexer.fit(df).transform(df)

# String indexer for cp_time
indexer = StringIndexer(inputCol="cp_time", outputCol="cp_time_cat")
# index cp_time in data
df1 = indexer.fit(df1).transform(df1)
df1 = df1.drop('cp_dose')
df1 = df1.drop('cp_time')

# One-hot enocder 
encoder = OneHotEncoder(inputCols=["cp_time_cat", "cp_dose_cat"],
                        outputCols=["cp_time_onehot", "cp_dose_onehot"])

model = encoder.fit(df1)
df1 = model.transform(df1)

# Keeping n-1 dummy variables for each feature. (dummy variables have degree of fredom n-1)
df1 = df1.withColumn("cp_time_cols", vector_to_array("cp_time_onehot")).select(df1.columns + [col("cp_time_cols")[i] for i in range(2)])
df1 = df1.withColumn("cp_dose_cols", vector_to_array("cp_dose_onehot")).select(df1.columns + [col("cp_dose_cols")[i] for i in range(1)])

# drop leftover cols
df1 = df1.drop('cp_dose_cat',
 'cp_time_cat',
 'cp_time_onehot',
 'cp_dose_onehot',
)


### Feature Engineering  

Third, we create some features that describes the statistics of each row. We look at the row-wise min, max, mean, standard deviation for gene and cell information. We also use KMeans clustering to classify the data into 3 cluster classes based on Euclidean distance and use the cluster results as an additional engineered feature.

In [ ]:
gene_feature_names = [name for name in df1.columns if 'g-' in name]
cell_feature_names =  [name for name in df1.columns if 'c-' in name]

df2 = df1.withColumn("gene_max", F.greatest(*gene_feature_names))
df2 = df2.withColumn("gene_min", F.least(*gene_feature_names))
df2 = df2.withColumn("cell_max", F.greatest(*cell_feature_names))
df2 = df2.withColumn("cell_min", F.least(*cell_feature_names))


df2 = df2.withColumn("gene_mean", reduce(lambda x,y: x+y, (col(x) for x in gene_feature_names)) / len(gene_feature_names))
df2 = df2.withColumn("cell_mean", reduce(lambda x,y: x+y, (col(x) for x in cell_feature_names)) / len(cell_feature_names))





### General Purpose Feature Selection Based on Correlation

In our initial step for feature selection we use correlation meassure to reduce the number of features.

In [9]:
def get_high_corr_features(df, gene_feature_names, cell_feature_names ):  # This function will be referenced by train_individual_label function below 
    
    # Keeping track of the feature names
    feature_columns = gene_feature_names + cell_feature_names 

    # Creating the feature vector
    vectorAssembler = VectorAssembler(inputCols = feature_columns, outputCol = 'feats' )
    feature_vector = vectorAssembler.transform(df).select("feats")

    # Computing the correlations with pySpark
    corr_matrix = Correlation.corr(feature_vector, "feats").head()[0]

    # Convert the correlation desne matrix and apply mask and to get the indicies where high correlations are observed
    # In here, I convert the correlation matrix to numpy, and then use numpy's mask to obtain the lower traingle of the
    # matrix. I used numpy becasue pyspark does not have mask.

    # Detection highly correlated features
    corr_Array = corr_matrix.toArray()
    masked_corr = np.ma.masked_where(np.triu(np.ones_like(corr_Array, dtype=bool)), corr_Array, copy=True)  
    idx_high_corr_feats = set(np.argwhere(abs(masked_corr) > 0.60)[:,0])  # Set threshold to 90%
    # Identify the column to drop 
    features_to_drop = np.array(feature_columns)[list(idx_high_corr_feats)].tolist()


    return features_to_drop

## Machine Learning Pipeline 

The task for Mechanisms of Action is formally considred a Multi-label classification problem. We are employing the Binary-Relevance approach for the problem at hand. For each of the possible binary labels we first use a **Random Forrest CLassifier** to achive another level of feature selection based on the feature importance meassure. Subsequently, we train one **logistic regression** model and a **naive bayes** model for each label.

We have use upsampling the minority class and downsampling the majority class to overcome issues of imbalanced data. 

In [ ]:
# The fucntion train_test_vectorize packs all the features into a single vector. This function was referred to in the train_individual_label function.

def train_test_vectorize(train, test, drop_features,  label_name):
    train  = train.drop(*drop_features)
    test = test.drop(*drop_features)
    # Getting the final feature set
    final_feature_names = list(set(train.columns) - set((label_name, )))

    # Create Feature vector
    vectorAssembler = VectorAssembler(inputCols = final_feature_names, outputCol = 'features' )

    # drop all unnecessary features
    train = vectorAssembler.transform(train).drop(*final_feature_names)
    test = vectorAssembler.transform(test).drop(*final_feature_names)
    
    
    
    train = train.select(F.col(label_name).alias('label'), F.col('features'))  # Just renaming these columns
    test = test.select(F.col(label_name).alias('label'), F.col('features')) # Just renaming these columns
    
    return train, test

In [9]:
list_of_label_names = df_label.columns
list_of_feature_names = list(set(df2.columns) - set(df_label.columns))


In [ ]:
df2.cache()
df2.show()

### Function for the tuned model

In [11]:


# This function is responsible for running our ml pipeline

#df is the train dataset, label_name is the label we want to do the training on

def train_individual_label(df, label_name, seed = 43, ):

    try:
    # we need at least 2 positive instances for this particular label_name
        if df.filter(df[label_name] == 1).count() >= 2:   # The model taining is only performed for labels with more than two activated samples

            # dataframe consisting only of the train features and the label name
            temp_df = df.select(*list_of_feature_names, label_name)

            # stratify split of the dataframe for train-test split
            fractions = {1: 0.8, 0: 0.8}
            train_df = temp_df.stat.sampleBy(label_name, fractions, )
            test_df =  temp_df.subtract(train_df)
            
            high_corr_names = get_high_corr_features(train_df, gene_feature_names, cell_feature_names )
            train_df, test_df  =  train_test_vectorize(train_df, test_df, high_corr_names,  label_name)

    

            # another layer of feature selection using random forrest before training the ml models
            clf = RandomForestClassifier(numTrees=20, maxDepth=5, featuresCol='features',  seed=42)
            model = clf.fit(train_df)
            feature_importance = model.featureImportances.toArray()

            # indeces of only the top 5% of features
            important_feature_idx = feature_importance.argsort()[-int(0.05 * len(feature_importance)):]  



            # Now, after getting the index, filter the feature vector based on the above feature importance index
            slicer = VectorSlicer(inputCol="features", outputCol="sub_features", indices=important_feature_idx)
            final_train_df_sub_feats =  slicer.transform(train_df).drop('features')
            final_test_df_sub_feats = slicer.transform(test_df).drop('features')

            final_train_df_sub_feats = final_train_df_sub_feats\
                                 .select(F.col('label'), F.col('sub_features').alias('features'))  # Just renaming these columns

            final_test_df_sub_feats = final_test_df_sub_feats\
                                 .select(F.col('label'), F.col('sub_features').alias('features'))  # Just renaming these columns




            # Use CV to train a logistic regression model
            lr = LogisticRegression(maxIter=10)
            lr_paramGrid = ParamGridBuilder() \
                            .addGrid(lr.regParam, [ 0.1, 0.01]) \
                            .addGrid(lr.elasticNetParam, [1,  0])\
                            .build()


            lr_evaluator = MulticlassClassificationEvaluator(metricName='logLoss')
            lr_crossval = StratifiedCrossValidator(estimator=lr,
                                  estimatorParamMaps=lr_paramGrid,
                                  evaluator=lr_evaluator,
                                  numFolds=3,
                                  parallelism=4)  

            lr_cvModel = lr_crossval.fit(final_train_df_sub_feats)

            lr_prediction_df = lr_cvModel.transform(final_test_df_sub_feats)

            ### In the code below, we are saving the trained data for future use.   ####

            lr_cvModel.save(f"./logistics_new/{label_name}.model")
            lr_prediction_df.write.save(f"./logistics_new/{label_name}_prediction_df.parquet", format="parquet")
            final_train_df_sub_feats.write.save(f"./logistics_new/{label_name}_train_df.parquet", format="parquet")

            lr_log_loss = lr_evaluator.evaluate(lr_prediction_df)

            with open(f"./logistics_new/log.log", 'a') as f:
                f.write('logistics, '+ str(label_name) + ', ' + str(lr_log_loss) + '\n') 


        else:   # 
            with open(f"./logistics_new/log.log", 'a') as f:
                f.write('logistics, '+ str(label_name) + ', ' + 'False' + '\n') 




    except Exception as e: # log the error as well; these are to be retrained later using a differnt seed later
        with open(f"./logistics_new/log.log", 'a') as f:
            f.write('logistics, '+ str(label_name) + ', ' + 'errored' + '\n') 


### Function for the untuned model below

In [ ]:
# This function is responsible for running our ml pipeline

#df is the train dataset, label_name is the label we want to do the training on
def train_individual_label_baseline(df, label_name, seed = 42,):
    try:
    
    # we need at least 2 positive instances for this particular label_name
        if df.filter(df[label_name] == 1).count() >= 2:  # Only the labels with activation numbers greater than 2 will be trained

            # dataframe consisting only of the train features and the label name
#             temp_df = df.select('feats', label_name)
            temp_df = df.select(*list_of_feature_names, label_name)

            # stratify split of the dataframe for train-test split
            fractions = {1: 0.8, 0: 0.8}
            train_df = temp_df.stat.sampleBy(label_name, fractions, seed, )
            test_df =  temp_df.subtract(train_df)


            
            high_corr_names = get_high_corr_features(train_df, gene_feature_names, cell_feature_names )
            train_df, test_df  =  train_test_vectorize(train_df, test_df, high_corr_names,  label_name)

            ################## over/down samping  ##################
            
            activation_samples = train_df.filter(train_df['label'] == 1)
            non_activation_samples = train_df.filter(train_df['label'] == 0)
            ratio = activation_samples.count() / non_activation_samples.count()

            default_down_sample_ratio = 0.5  # This can be changed, but for now,i am just setting it as 0.5

            upsample_ratio = default_down_sample_ratio / ratio

            activation_samples_up = activation_samples.sample(True, upsample_ratio, 42)   # Upsample the activation samples

            non_activation_samples_down = non_activation_samples.sample(True, default_down_sample_ratio, 42)  # Meanwhile, downsampling the non-activated samples 


            train_df = activation_samples_up.union(non_activation_samples_down).orderBy(F.rand())
            
            
            ###########################################


            # another layer of feature selection using random forrest before training the ml models
            clf = RandomForestClassifier(numTrees=20, maxDepth=5, featuresCol='features',  seed=42)
            model = clf.fit(train_df)
            feature_importance = model.featureImportances.toArray()

            # indeces of only the top 5% of features
            important_feature_idx = feature_importance.argsort()[-int(0.05 * len(feature_importance)):]  



            # Now, after getting the index, filter the feature vector based on the above feature importance index
            slicer = VectorSlicer(inputCol="features", outputCol="sub_features", indices=important_feature_idx)
            final_train_df_sub_feats =  slicer.transform(train_df).drop('features')
            final_test_df_sub_feats = slicer.transform(test_df).drop('features')

            final_train_df_sub_feats = final_train_df_sub_feats\
                                 .select(F.col('label'), F.col('sub_features').alias('features'))  # Just renaming these columns

            final_test_df_sub_feats = final_test_df_sub_feats\
                                 .select(F.col('label'), F.col('sub_features').alias('features'))  # Just renaming these columns




            # Use CV to train a logistic regression model
            lr = LogisticRegression(maxIter=10)



            lr_evaluator = MulticlassClassificationEvaluator(metricName='logLoss')


            lr_Model = lr.fit(final_train_df_sub_feats)

            lr_prediction_df = lr_Model.transform(final_test_df_sub_feats)

            lr_Model.save(f"./logistics-baseline-new/{label_name}.model")
            lr_prediction_df.write.save(f"./logistics-baseline-new/{label_name}_prediction_df.parquet", format="parquet")
            final_train_df_sub_feats.write.save(f"./logistics-baseline-new/{label_name}_train_df.parquet", format="parquet")

            lr_log_loss = lr_evaluator.evaluate(lr_prediction_df)

            with open(f"./logistics-baseline-new/log.log", 'a') as f:
                f.write('logistics, '+ str(label_name) + ', ' + str(lr_log_loss) + '\n') 

        else:
            with open(f"./logistics-baseline-new/log.log", 'a') as f:
                f.write('logistics, '+ str(label_name) + ', ' + 'False' + '\n') 
         

    except:
        with open(f"./logistics-baseline-new/log.log", 'a') as f:
            f.write('logistics, '+ str(label_name) + ', ' + 'errored' + '\n') 


# Training

The code below  runs the `train_individual_label` function for each label, to train the **tuned** model

In [ ]:
for name in tqdm(df_label.columns[1:]):
    if not os.path.exists(f'./logistics_new/{name}.model'):
        train_individual_label(df2,  name,)


The code below  runs the `train_individual_label_baseline` function for each label, to train the **untuned** model

In [ ]:
for name in tqdm(df_label.columns[1:]):
    if not os.path.exists(f'./logistics_new/{name}.model'):
        train_individual_label_baseline(df2,  name, )
